In [179]:


####

clusters = [
            ("QA (iotqa.arcgis.com)","https://iotqa.arcgis.com/"),
            ("Dev_Demo (iotdev.arcgis.com)","https://iotdev.arcgis.com/"),
            ("USP1 (us-iotdevops.arcgis.com)","https://us-iotdevops.arcgis.com/"),
            ("DSNY (a4iot-services-dsny.arcgis.com)","https://a4iot-services-dsny.arcgis.com/"),
            ("ESS (iot.arcgis.com)","https://iot.arcgis.com/")
           ]


#### set downloadDir to specify the folder on the local system where the script will save configuration files. 
#### If left blank it will default to the folder where this script is located. 
#### backslashes require an escape character. 
#### Example: to specify the path "C:\\A4iot\backups" the downloadDir would be "C:\\\A4iot\\backups"
downloadDir = ""

#### set backupsDir to specify the folder on the local system where the script will look for 
#### previously exported configuration files to deploy. This can be set to the same folder as downloadDir but does not have to be.
#### If left blank it will default to the folder where this script is located. 
#### backslashes require an escape character. 
#### Example: to specify the path "C:\\A4iot\backups" the backupsDir would be "C:\\\A4iot\\backups"
backupsDir = ""

#### When deploying a configuration, the script will create feeds and analytics for this user in the destination organization. 
#### If the user already exists it will first delete all feeds and analytics for the user.
#### If the user does not exist, it will be created using the following deployUser_ settings 
#### It will then deploy the feeds and analytics defined in the selected configuration to the user's profile.
deployUser_Username = "certify_a4iot"
deployUser_Password = "certify.a4iot"
deployUser_Firstname = "Demo"
deployUser_Lastname = "User"
deployUser_Email = "kgorton@esri.com"
deployUser_Description = "IoT user account for use in demos"
deployUser_Role = "org_iot_publisher"













#Do not edit below this line
##########################################################################


import ipywidgets as widgets
from IPython.display import display
import requests 
import json
import getpass
from arcgis.gis import GIS
from arcgis.gis import RoleManager
import os


try:
    actions = [('Export a backup of my A4IoT feeds and analytics','export from'),
               ('Deploy my backed up feeds and analytics to a new A4IoT cluster','deploy to')]

    if not downloadDir:
        downloadDir = os.getcwd()
    if not backupsDir:
        backupsDir = os.getcwd()
    if not deployUser_Username:
        raise Exception("Set the deployUser_Username value in the script settings section.")        
    if not deployUser_Password:
        raise Exception("Set the deployUser_Password value in the script settings section.")
    if not deployUser_Firstname:
        raise Exception("Set the deployUser_Firstname value in the script settings section.")
    if not deployUser_Lastname:
        raise Exception("Set the deployUser_Lastname value in the script settings section.")
    if not deployUser_Email:
        raise Exception("Set the deployUser_Email value in the script settings section.")
    if not deployUser_Description:
        deployUser_Description = "IoT user account for use in demos"
    if not deployUser_Role:
        deployUser_Role = "IoT Publisher"

    jsonfiles = []


    for file in os.listdir(backupsDir):
        filename, file_extension = os.path.splitext(file)
        if filename[:13] == "A4IoT Backup " and file_extension == ".json":
            jsonfiles.append(file)


    actionList = widgets.RadioButtons(
        options=actions,
        value=None,
        description='I want to...:',
        disabled=False,
        layout=widgets.Layout(width='80%')
    )

    filesList = widgets.Dropdown(
        options=jsonfiles,
        value=None,
        description='Backup:',
        layout=widgets.Layout(width='80%',visibility='hidden')
    )

    clusterList = widgets.Dropdown(
        options=clusters,
        value=None,
        description='Cluster:',
        disabled=False,
        layout=widgets.Layout(width='50%')
    )

    userNameText = widgets.Text(
        description='Username:',
        layout=widgets.Layout(width='50%')
    )

    passwordText = widgets.Password(    
        description='Password:',
        layout=widgets.Layout(width='50%')
    )

    actionButton = widgets.Button(
        description='Go',
        disabled=False,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
    )

    def actionList_Change(change):
        if actionList.value == 'deploy to':
            filesList.layout.visibility='visible'
        else:
            filesList.layout.visibility='hidden'

    actionList.observe(actionList_Change)

    def exportUserContent(apiUrl, userToken):
        response = requests.get(url = apiUrl+"/iot/config/export", params = {'token':userToken,"f": "pjson"})     
        print(response.json())


    def actionButtonEventHandler(b):
        out.clear_output()

        a4iotUrl = clusterList.value
        portalUrl = ""
        if a4iotUrl == "https://iot.arcgis.com/":
            portalUrl = "https://www.arcgis.com/sharing/rest"        
        else:
            # get the portal url from the A4IoT url
            hed = {"Content-Type": "application/x-www-form-urlencoded"}
            req = requests.get(url = a4iotUrl+"/env.json", headers = hed)
            response = req.json()
            portalUrl = response["portal"]  

        if not portalUrl:
            raise Exception("Could not obtain the portal URL for authentication purposes.")

        # ask for the user's credentials and use them to request a token from the portalUrl
        adminUser = userNameText.value
        adminPassword = passwordText.value    
        
        gis = GIS(portalUrl, username=adminUser, password=adminPassword ) 
        
        #rm = RoleManager(gis)
        #roles = rm.all()
        #with out:
        #    print(roles)
                   
        orgC = gis._subscription_information["orgCapabilities"]
        iotBaseURL = ""
        for c in orgC:
            if "analytics4IoT" == c["id"] and "Active" == c["status"]:
                iotBaseURL = c["iotRegionUrl"] + "/iot"                
        token = gis._con.token

        with out:
            print("\nLogged in to " + a4iotUrl + " as " + adminUser+ ".")   


        if actionList.value == 'export from':  
            with out:
                print('\nPreparing to export the feeds and analytics for ' + adminUser + '. Please be patient as this may take a few moments.')
            response = requests.get(url = iotBaseURL+"/config/export", params = {'token':token,"f":"pjson"}) 
            filename = "A4IoT Backup " + clusterList.label + " " + adminUser + ".json"
            downloadpath = os.path.join(downloadDir, filename)
            #print(response)
            with open(downloadpath, 'w', encoding='utf-8') as outfile:
                json.dump(response.json(), outfile, ensure_ascii=False, indent=4)
            outfile.close()
            with out:
                print("\nYour feeds and analytics have been exported to:")            
                print("\t" + downloadpath)
            if filename not in jsonfiles:
                jsonfiles.append(filename)
            filesList.options = jsonfiles

        if actionList.value == 'deploy to':

                # check for the user in the org                
                cUsers = gis.users.get(username=deployUser_Username)

                if cUsers == None:   
                    with out:
                        print("\nCreating user: " + deployUser_Username)
                    tempPassword = 'deploy.passtemp4'
                    demo_user1 = gis.users.create(username = deployUser_Username,
                                              password = tempPassword,
                                              firstname = deployUser_Firstname,
                                              lastname = deployUser_Lastname,
                                              email = deployUser_Email,
                                              description = deployUser_Description,
                                              role = deployUser_Role,
                                              level = 2,
                                              provider = 'arcgis')
                    demo_user1.reset(password=tempPassword, 
                                     new_password=deployUser_Password)   
                    
                cUsers = gis.users.get(username=deployUser_Username)
                #todo verify user has privileges to create and own A4IoT content
                with out:
                    print("Role: " + cUsers.role)
                    print("Level: " + cUsers.level)


                gis2 = GIS(portalUrl, username=cUsers.username, password=deployUser_Password )
                token2 = gis2._con.token

                with out:
                    print("\nDeploying content to user " + cUsers.username)


                #todo clear all existing feeds and analytics fromthe user's content
                #response = requests.get(url = iotBaseURL+"/config/reset", params = {'token':token2,"f": "pjson"})
                #print("deleting old stuff")
                folders = cUsers.folders
                for item in cUsers.items(folder=None, max_items=1000):
                        item.delete()
                #print("deleted some old stuff")
                
                for folder in folders:
                    for item in cUsers.items(folder=folder, max_items=1000):
                        item.delete()
                
                #print("deleted more old stuff")



                # read the backup file
                jsonFile = os.path.join(backupsDir, filesList.value)
                config = ""
                with open(jsonFile) as f:
                    config = json.load(f)  
                f.close()
                
                if not config:
                    raise Exception("Could not open or read " + jsonFile)

                # for each feed in the backup file deploy it to the new cluster and
                # keep a running list of the old guid and new guid for each one
                feeds = config['feeds']

                guids = {}
                reqHeader = {'accept':'application/json',
                                     'Content-Type':'application/json',
                                     'Authorization':'token='+token2,
                                     'Referer':'http://localhost:8888'}
                if len(feeds) > 0:
                    with out:
                        print("\nDeploying feeds...")
                    for feed in feeds: 
                        reqUrl = iotBaseURL+"/feed/"                        
                        reqData = json.dumps(feed)
                        req = requests.post(url = reqUrl, headers = reqHeader, data = reqData)
                        response = req.json()
                        if req.status_code == 200:
                            guids[feed["id"]]=response["id"]
                            with out:
                                print("\t" + feed["label"] +" deployed OK.")
                        else:
                            with out:
                                print("\tFailed to deploy " + feed["label"] + ": " + req.text)


                # for each RAT in the backup file, for each feed in the RAT, 
                # fetch the feed's new guid from the list above and replace all instances in the RAT of the old guid
                # with the new guid. Then deploy the updated RAT to the new cluster  
                rats = config['realtimeanalytics']
                if len(rats) > 0:
                    with out:
                        print("\nDeploying real-time analytics...")
                    for rat in rats:
                        reqUrl = iotBaseURL+"/analytics/realtime/"      

                        reqData = json.dumps(rat)
                        for guidPair in guids:                            
                            reqData = reqData.replace(guidPair,guids[guidPair])

                        req = requests.post(url = reqUrl, headers = reqHeader, data = reqData)
                        response = req.json()                            
                        if req.status_code == 200:
                            with out:
                                print("\t" + rat["label"] +" deployed OK.")
                        else:
                            with out:
                                print("\tFailed to deploy " + rat["label"] + ": " + req.text)




                # for each BAT in te backup file, deploy it to the new cluster
                bats = config['batchanalytics']
                if len(bats) > 0:
                    with out:
                        print("\nDeploying big data analytics...")
                    for bat in bats:
                        reqUrl = iotBaseURL+"/analytics/bigdata/"      

                        reqData = json.dumps(bat)

                        req = requests.post(url = reqUrl, headers = reqHeader, data = reqData)
                        response = req.json()                            
                        if req.status_code == 200:
                            with out:
                                print("\t" + bat["label"] +" deployed OK.")
                        else:
                            with out:
                                print("\tFailed to deploy " + bat["label"] + ": " + req.text)


                #f.close()
                with out:
                    print("\nDone")
except RuntimeError:
    with out:
        print("Error")
except:
    with out:
        print("Error")

            
    
actionButton.on_click(actionButtonEventHandler)

out = widgets.Output(layout={'border': '1px solid black'})
widgets.VBox([actionList, filesList, clusterList, userNameText,passwordText,actionButton,out])

